### Compare epsilon to Lombardo&Gregg epsilon

In [2]:
%matplotlib inline

import sys
sys.path.append('../src/')

import glob
import warnings

import numpy as np
import pandas as pd
import xarray as xr

from scipy.optimize import curve_fit, minimize

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from cmocean import cm
import hvplot.xarray

from sklearn.linear_model import LinearRegression

from epsilon_tools import *
from tools import str2date, avg_funs

import warnings
warnings.simplefilter("ignore",category=FutureWarning)
warnings.simplefilter("ignore",category=RuntimeWarning)

# set up figure params
sns.set(style='ticks', context='poster', palette='colorblind')
mpl.rc('figure', dpi=100, figsize=[12, 6])
mpl.rc('savefig', dpi=500, bbox='tight')
mpl.rc('legend', frameon=False)

### data wrangling

In [3]:
dat = xr.open_dataset('../data/ml/ww_7781a_1h_2Tf.nc')
met = xr.open_dataset('../data/xarray/qc_ww_7781a.nc')
dat = dat.sel( time = slice('9-15-2016','12-10-2016') )
met = met.sel( mettime = slice('9-15-2016','12-10-2016') )
met = met.sel( time = slice('9-15-2016','12-10-2016') )
emp = xr.open_dataset('../data/CFS/croppped_emp_field.nc')

In [4]:
met['S2'] = np.sqrt(met.dudz**2 + met.dvdz**2)

In [5]:
# met.S2.plot()
# met.S2.max(dim='z').plot(color='w')
met['mld_new'] = ('time', met.z.where( met.S2 == met.S2.max(dim='z')).mean(dim='z').rolling(time=6, center=True).mean())

In [6]:
met['S0'] = ('time',met.S.where(met.z>met.mld_new).mean(dim='z'))
met['epz'] = ('time',met.eps.where(met.z>met.mld_new).mean(dim='z'))

In [7]:
flot = xr.open_dataset('../data/xarray/qc_7781a.nc')
import gsw
cp = gsw.cp_t_exact(flot.isel(z=2).S.mean(),flot.isel(z=2).T.mean(),0)
alphaT = gsw.alpha(flot.isel(z=2).S.mean(),flot.isel(z=2).T.mean(),0)
betaS = gsw.beta(flot.isel(z=2).S.mean(),flot.isel(z=2).T.mean(),0)

In [8]:
rho0=1025
g = 9.81
met['tau'] = 0.5*np.sqrt( met.tx**2 + met.ty**2 )
met['uprime'] = (np.sqrt( met.tau/rho0))**3

met['mld_new'] = ('mettime', met.mld_new.dropna('time').interp(time=met.mettime) )
met['mld'] = ('mettime', met.mld.dropna('time').interp(time=met.mettime) )
met['S0'] = ('mettime', met.S0.dropna('time').interp(time=met.mettime) )
met['epz'] = ('mettime', met.epz.dropna('time').interp(time=met.mettime) )
met['lon'] = ('mettime', met.lon.dropna('time').interp(time=met.mettime) )
met['lat'] = ('mettime', met.lat.dropna('time').interp(time=met.mettime) )

In [9]:
resample = emp.sortby('time').resample(time='1h').mean()

In [10]:
resample = resample.rename({'time':'mettime'})
sub = resample.EMP.interp_like(met) # interpolates onto floats time

In [11]:
collect = []
for t in sub.mettime:
    temp1 = sub.sel(mettime=t)
    temp2 = met.sel(mettime=t)
    
    if ~temp2.lon.isnull() & ~temp2.lat.isnull():
        collect.append(temp1.interp(lon=temp2.lon,lat=temp2.lat))
#     else:
#         collect.append(np.nan)
# sub.interp()

In [12]:
collect = xr.concat(collect,dim='mettime')
met['emp'] = collect
met['emp'] = met.emp/(100*86400)

In [13]:
met['BQ'] = g*alphaT*met.Qnet/(cp*rho0)
met['BEP'] = g*betaS*met.emp*met.S0/(1-met.S0/1000)
met['B'] = met.BQ + met.BEP
met['MO'] = -met.uprime/(0.4*met.B)
met['MO_MLD'] = met.MO/met.mld_new
met['uprime_kz'] = met.uprime/(0.4*met.mld_new)

### compare L&G prediction to measured $\epsilon$

Lombardo & Gregg scaling:

If $0 < -z \ll -L $,
$$
\epsilon_s = \frac{u^{*3}}{\kappa z}.
$$
If $-L \ll -z \ll D$,
$$
\epsilon_f = J_b^0.
$$
If $-L \ll -z < D$,
$$
\epsilon_{ML} = J_b^0.
$$